In [90]:
# работа с изображениями и текстом, пропустить
from PIL import Image, ImageDraw, ImageFont

# create an image
out = Image.new("RGB", (150, 100), (255, 255, 255))

# get a font
# get a drawing context
d = ImageDraw.Draw(out)
d.line([0,0, 80,80], fill=128)
# draw multiline text
d.multiline_text((10,10), "Hello\nWorld", font=d.getfont(), fill=(0, 200, 0))
out.save('newImage.png', 'png')

In [2]:
#загрузка окра, при смене документа запускать
import urllib3
import json
import config
http = urllib3.PoolManager()
r = http.request('GET', config.ocr_href)
ocrData = r.data.decode('utf-8')
f = open('./tmp/ocrData.csv', 'w')
f.write(ocrData)
f.close()

In [1]:
# обновление файла с картинкой (очистка всего что нарисовалось локально)
import urllib3
import config
http = urllib3.PoolManager()
r = http.request('GET', config.image_href)
f = open(config.image_name, 'wb')
f.write(r.data)
f.close()


In [3]:
import numpy as np
import pandas as pn
import config
import math
from PIL import Image, ImageDraw, ImageFont

frameOcr = pn.read_csv('./tmp/ocrData.csv')
# точка начала базиса
OnePoint = frameOcr.query("text == 'Указания'")
# вспомогательные вектора (в принципе достаточно одного, для масштабирования, использую А)
APoint = frameOcr.query("text == 'Прием'")
BPoint = frameOcr.query("text == 'Сдача'")


with Image.open("test.png") as docImage:

    draw = ImageDraw.Draw(docImage)
    basis = config.basis
    ofsetX = OnePoint.iloc[0]['left']
    ofsetY = OnePoint.iloc[0]['top']
    # scale
    documentScaleX = (BPoint.left - ofsetX) / basis['B']['x']
    documentScaleY = (BPoint.top - ofsetY) / basis['B']['y']
    # point coords    
    BasisPosition = [ofsetX, ofsetY]
    
    for point in basis:
        vector = basis[point]
        vector = [vector['x'] * documentScaleX + ofsetX, vector['y'] * documentScaleY + ofsetY]
        draw.line(BasisPosition + vector, fill=128, width=10)
    docImage.save("test.png", "PNG")